In [2]:
import numpy as np
import sys
from qiskit.providers.aer import QasmSimulator
from qiskit.algorithms import QAOA,VQE
from qiskit_optimization.algorithms import MinimumEigenOptimizer    
from qiskit.quantum_info.operators import Operator
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.utils import algorithm_globals, QuantumInstance
# useful additional packages
from qiskit import Aer,QuantumCircuit
from qiskit.circuit import Parameter
sys.path.insert(1,"C:\\Users\\malak\\FlowShop_QuantumAlgorithms\\VQA")
from fspmodel.timeIndex import FSPTimeIndexform
from fspmodel.FSPasTSP import FSPasTSP

In [2]:
spsa = SPSA(maxiter=5000)
ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
device = QuantumInstance(QasmSimulator(method='matrix_product_state'), shots=600)
vqe = VQE(ansatz, optimizer=spsa, quantum_instance=device)
qaoa = QAOA(optimizer=spsa, reps=1, quantum_instance=device,include_custom=True)

In [3]:
from qiskit.circuit import Parameter
from qiskit.extensions import UnitaryGate
import numpy as np
from qiskit import QuantumCircuit
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import Operator
import math as m


from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, Gate


class UHGate(Gate):
    def __init__(self, theta, label=None):
        super().__init__('U', 2, [theta], label=label)
        
        
    def _define(self):
        qc = QuantumCircuit(2)
        qc.unitary(self.to_matrix(), [0, 1])
        
        self.definition = qc
        
    def to_matrix(self):
        t = float(self.params[0])
        a = np.cos( t)+1j * np.sin( t)
        c = np.cos( t)
        s = 1j * np.sin( t)

        return np.array([[a, 0, 0, 0], 
                         [0, c, s, 0], 
                         [0, s, c, 0],
                         [0, 0, 0, a]])


t = Parameter('θ')
qc = QuantumCircuit(4)
qc.append(UHGate(t), [0, 2])
qc.append(UHGate(t), [1,3])

In [4]:
def init_point():
    qc = QuantumCircuit(4)
    qc.x(0)
    qc.x(3)
    return qc

In [5]:
qaoa = QAOA(optimizer=spsa,
            reps=1,
            quantum_instance=device,
            initial_state= init_point(),
            mixer=qc,
            initial_point=[0.0, 1.0],
            include_custom=True)

In [6]:
fsp = FSPasTSP(2,[[6, 3], [7, 7]],2,1)
operator,off = fsp.to_ising()
result1 = qaoa.compute_minimum_eigenvalue(operator)
#result1
"""optimizer2 = MinimumEigenOptimizer(vqe)
result2 = optimizer2.solve(fsp.to_qubo())"""

'optimizer2 = MinimumEigenOptimizer(vqe)\nresult2 = optimizer2.solve(fsp.to_qubo())'

In [7]:
result1.eigenstate


{'0011': 0.469041575982343,
 '0110': 0.689202437604511,
 '1001': 0.32145502536643183,
 '1100': 0.4490731195102493}

In [8]:
qaoa2 = QAOA(optimizer=spsa,
            reps=1,
            quantum_instance=device,
            initial_point=[0.0, 1.0],
            include_custom=True)
result2 = qaoa2.compute_minimum_eigenvalue(operator)
result2.eigenstate

{'0000': 0.2516611478423583,
 '0001': 0.2449489742783178,
 '0010': 0.2345207879911715,
 '0011': 0.23804761428476165,
 '0100': 0.2645751311064591,
 '0101': 0.2516611478423583,
 '0110': 0.21984843263788198,
 '0111': 0.2273030282830976,
 '1000': 0.2516611478423583,
 '1001': 0.2516611478423583,
 '1010': 0.24152294576982397,
 '1011': 0.26770630673681683,
 '1100': 0.29154759474226505,
 '1101': 0.248327740429189,
 '1110': 0.27080128015453203,
 '1111': 0.2345207879911715}

In [9]:
def init_point_supp():
    qc = QuantumCircuit(4)
    qc.h(0)
    qc.x(0)
    qc.cx(0,1)
    qc.cx(0,2)
    qc.x(0)
    qc.x(1)
    qc.cx(1,3)
    qc.x(1)
    return qc

In [10]:
qaoa3 = QAOA(optimizer=spsa,
            reps=1,
            quantum_instance=device,
            initial_state= init_point_supp(),
            mixer=qc,
            include_custom=True)
result3 = qaoa3.compute_minimum_eigenvalue(operator)
result3.eigenstate

{'0110': 0.714142842854285, '1001': 0.7}

# Generalisation N jobs

In [4]:
def init_state(n):
    qc = QuantumCircuit(n*n)
    for i in range(n):
        qc.x(i*n+i)
    return qc


In [5]:
from qiskit.circuit import ParameterVector
n = 4
N = 16
t = ParameterVector('θ',(N//n-1))
qc = QuantumCircuit(N)
qubits_pair = []
for i in range(N//n):
    for j in range(n-1):
        qubits_pair.append((t[j],i,(j+1)*3+i))
print(qubits_pair)
for i,pair in enumerate(qubits_pair):
    print(pair)
    qc.append(UHGate(pair[0]), [pair[1],pair[2]])
qc.draw()        

[(ParameterVectorElement(θ[0]), 0, 3), (ParameterVectorElement(θ[1]), 0, 6), (ParameterVectorElement(θ[2]), 0, 9), (ParameterVectorElement(θ[0]), 1, 4), (ParameterVectorElement(θ[1]), 1, 7), (ParameterVectorElement(θ[2]), 1, 10), (ParameterVectorElement(θ[0]), 2, 5), (ParameterVectorElement(θ[1]), 2, 8), (ParameterVectorElement(θ[2]), 2, 11), (ParameterVectorElement(θ[0]), 3, 6), (ParameterVectorElement(θ[1]), 3, 9), (ParameterVectorElement(θ[2]), 3, 12)]
(ParameterVectorElement(θ[0]), 0, 3)
(ParameterVectorElement(θ[1]), 0, 6)
(ParameterVectorElement(θ[2]), 0, 9)
(ParameterVectorElement(θ[0]), 1, 4)
(ParameterVectorElement(θ[1]), 1, 7)
(ParameterVectorElement(θ[2]), 1, 10)
(ParameterVectorElement(θ[0]), 2, 5)
(ParameterVectorElement(θ[1]), 2, 8)
(ParameterVectorElement(θ[2]), 2, 11)
(ParameterVectorElement(θ[0]), 3, 6)
(ParameterVectorElement(θ[1]), 3, 9)
(ParameterVectorElement(θ[2]), 3, 12)


┌──────────┐                        ┌──────────┐                        »
 q_0: ┤0         ├────────────────────────┤0         ├────────────────────────»
      │          │┌──────────┐            │          │┌──────────┐            »
 q_1: ┤          ├┤0         ├────────────┤          ├┤0         ├────────────»
      │  U(θ[0]) ││          │┌──────────┐│          ││          │┌──────────┐»
 q_2: ┤          ├┤          ├┤0         ├┤          ├┤          ├┤0         ├»
      │          ││  U(θ[0]) ││          ││          ││          ││          │»
 q_3: ┤1         ├┤          ├┤          ├┤  U(θ[1]) ├┤          ├┤          ├»
      └──────────┘│          ││  U(θ[0]) ││          ││          ││          │»
 q_4: ────────────┤1         ├┤          ├┤          ├┤  U(θ[1]) ├┤          ├»
                  └──────────┘│          ││          ││          ││          │»
 q_5: ────────────────────────┤1         ├┤          ├┤          ├┤  U(θ[1]) ├»
                              └──────────┘│          ││          ││          │»
 q_6: ────────────────────────────────────┤1         ├┤          ├┤          ├»
                                          └──────────┘│          ││          │»
 q_7: ────────────────────────────────────────────────┤1         ├┤          ├»
                                                      └──────────┘│          │»
 q_8: ────────────────────────────────────────────────────────────┤1         ├»
                                                                  └──────────┘»
 q_9: ────────────────────────────────────────────────────────────────────────»
                                                                              »
q_10: ────────────────────────────────────────────────────────────────────────»
                                                                              »
q_11: ────────────────────────────────────────────────────────────────────────»
                                                                              »
q_12: ────────────────────────────────────────────────────────────────────────»
                                                                              »
q_13: ────────────────────────────────────────────────────────────────────────»
                                                                              »
q_14: ────────────────────────────────────────────────────────────────────────»
                                                                              »
q_15: ────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌──────────┐                                                            
« q_0: ┤0         ├────────────────────────────────────────────────────────────
«      │          │┌──────────┐                                                
« q_1: ┤          ├┤0         ├────────────────────────────────────────────────
«      │          ││          │┌──────────┐                                    
« q_2: ┤          ├┤          ├┤0         ├────────────────────────────────────
«      │          ││          ││          │┌──────────┐┌──────────┐┌──────────┐
« q_3: ┤          ├┤          ├┤          ├┤0         ├┤0         ├┤0         ├
«      │          ││          ││          ││          ││          ││          │
« q_4: ┤          ├┤          ├┤          ├┤          ├┤          ├┤          ├
«      │  U(θ[2]) ││          ││          ││  U(θ[0]) ││          ││          │
« q_5: ┤          ├┤          ├┤          ├┤          ├┤          ├┤          ├
«      │          ││  U(θ[2]) ││          ││          ││          ││          │
« q_6: ┤          ├┤          ├┤          ├┤1         ├┤  U(θ[1]) ├┤          ├
«      │          ││          ││  U(θ[2]) │└──────────┘│          ││          │
« q_7: ┤          ├┤          ├┤          ├────────────┤          ├┤          ├
«      │          ││          ││          │            │          ││  U(θ[2]) │
« q_8:

In [13]:
qaoa_sw = QAOA(optimizer=spsa,
            reps=1,
            quantum_instance=device,
            initial_state= init_state(n),
            mixer=qc,
            include_custom=True)

In [14]:
fsp = FSPasTSP(2,[[6, 3,5,1], [7,2,9,3]],4,1)
operator,off = fsp.to_ising()
result1 = qaoa_sw.compute_minimum_eigenvalue(operator)

In [15]:
result1.eigenstate

{'1000010000100001': 1.0}

In [ ]:
result1 = qaoa2.compute_minimum_eigenvalue(operator)